Author: Ningxin Kang (nik010@ucsd.edu)

Last Update: 10/11/2022

# Dataset Cleaning

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xlrd
import re

In [85]:
def merging_info(dir_file_GFP,dir_file_DAPI,area_filter_low,area_filter_high, dir_GFP_output,dir_DAPI_output):
    
    #########################
    # Importing the dataset #
    #########################
    # Import the data
    file_GFP = pd.read_excel(dir_file_GFP)
    file_DAPI = pd.read_excel(dir_file_DAPI)

    # Create two new dataframe for storing the data
    df_GFP = pd.DataFrame(columns=["Label", "Area",	"Mean",	"Mode",	"Min", "Max", "IntDen", "RawIntDen"])
    df_DAPI = pd.DataFrame(columns=['Slice', 'Count', 'Total Area', 'Average Size', 'Area Fraction', 'Mean'])

    # Read file and concate by line for BHB group
    for file in file_GFP['filepath']:
        df_GFP = pd.merge(df_GFP, pd.read_excel((file)), how = "outer")

    # Read file and concate by line for BHB group
    for file in file_DAPI['filepath']:
        df_DAPI = pd.merge(df_DAPI, pd.read_csv((file), sep = "\t"), how = "outer")

    # Change column name so they two dataframes matches
    df_DAPI.rename({'Slice': 'Label', "Mean": 'TotalMean'}, axis=1, inplace=True)
    
    ########################
    # Formating the df_GFP #
    ########################
    # temporary, for this dataset only, to format the label
    df_GFP['Label'] = [label.replace("GFP100MS", "GFP_100ms") for label in df_GFP['Label']]
    df_GFP['Label'] = [label.replace("6.5_CTRL", "CTRL_6.5") for label in df_GFP['Label']]
    df_GFP['Label'] = [label.replace("6.5_BHB", "BHB_6.5") for label in df_GFP['Label']]
    # scale the area from square cemtimeter tro square micrometer
    df_GFP["Area"] = [area*10e8 for area in df_GFP["Area"]]
    # Create a column recording the treatment of each sample
    df_GFP['category'] = [re.search('^((?:[^_]*\_){2})([^_]*)',df_GFP['Label'][i]).group(2) for i in range(df_GFP.shape[0])]
    # Extract the id of mouse from column 'Label'
    df_GFP['mouse_id'] = [re.search('LG.*_.*_.*[BL]',df_GFP['Label'][i]).group() for i in range(df_GFP.shape[0])]
    # Extract the if of picture from column 'Label'
    df_GFP['picture_id'] = [re.search('^((?:[^_]*\_){8})([^_]*)',df_GFP['Label'][i]).group(2)
                            for i in range(df_GFP.shape[0])]
    df_GFP['picture_id'] = df_GFP['mouse_id'] + '_' + df_GFP['picture_id']      # combine mouth id and picture id

    # change colname
    df_GFP.columns.values[0] = "sample_index"

    # Filter the aggregates by area
    df_GFP = df_GFP[df_GFP.Area > area_filter_low]
    df_GFP = df_GFP[df_GFP.Area < area_filter_high]

    #########################
    # Formating the df_DAPI #
    #########################
    # temporary, for this dataset only, to format the label
    df_DAPI['Label'] = [label.replace("GFP100MS", "GFP_100ms") for label in df_DAPI['Label']]
    df_DAPI['Label'] = [label.replace("6.5_CTRL", "CTRL_6.5") for label in df_DAPI['Label']]
    df_DAPI['Label'] = [label.replace("6.5_BHB", "BHB_6.5") for label in df_DAPI['Label']]
    # Extract the id of mouse from column 'Label'
    df_DAPI['mouse_id'] = [re.search('LG.*_.*_.*[BL]',df_DAPI['Label'][i]).group() for i in range(df_DAPI.shape[0])]
    # Extract the if of picture from column 'Label'
    df_DAPI['picture_id'] = [re.search('^((?:[^_]*\_){8})([^_]*)',df_DAPI['Label'][i]).group(2)
                            for i in range(df_DAPI.shape[0])]
    df_DAPI['picture_id'] = df_DAPI['mouse_id'] + '_' + df_DAPI['picture_id']      # combine mouth id and picture id
    df_DAPI = df_DAPI.drop(columns=['mouse_id',"Label"])
    
    #############
    # Exporting #
    #############
    df_GFP.to_csv(dir_GFP_output,sep = ",",index = False)
    df_DAPI.to_csv(dir_DAPI_output,sep = ",",index = False)
    
    return("Finish Cleaning!!")

In [86]:
dir_file_GFP = "/Users/sylvia618/Desktop/Chen Lab/hek cells/input/BHBDiet_allfiles/filepath.xls"
dir_file_DAPI = "/Users/sylvia618/Desktop/Chen Lab/hek cells/input/DAPI/filepath.xls"
dir_GFP_output = "/Users/sylvia618/Desktop/Chen Lab/hek cells/input/GFP.csv"
dir_DAPI_output = "/Users/sylvia618/Desktop/Chen Lab/hek cells/input/DAPI.csv"
area_filter_low = 0.416
area_filter_high = 150
merging_info(dir_file_GFP,dir_file_DAPI,area_filter_low,area_filter_high, dir_GFP_output,dir_DAPI_output)

/Users/sylvia618/miniconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:1108: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,
/Users/sylvia618/miniconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:1097: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


'Finish Cleaning!!'